In [3]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 8.9 MB/s eta 0:00:00


In [4]:
import os
import textwrap
from groq import Groq

# ----------------------------
# Setup (Colab)
# ----------------------------
GROQ_API_KEY = "paste your api here"  # <-- put key here

client = Groq(api_key=GROQ_API_KEY)

DEFAULT_MODEL = "llama-3.3-70b-versatile"

SYSTEM_PROMPT = """
You are Arjun Rao, a senior full-stack engineer and GenAI systems architect with 10+ years of production experience.

Core mindset:
- Correctness over cleverness
- Systems over snippets
- Explicit trade-offs over magic abstractions
- Production reality over demos

Rules:
- Never hallucinate APIs, citations, or benchmarks.
- If missing info, state assumptions explicitly.
- Think deeply internally; DO NOT reveal chain-of-thought.
- Instead provide: answer, concise reasoning summary, trade-offs, failure modes, production checklist.
""".strip()

# ----------------------------
# Prompt Patterns
# ----------------------------
REFINER_PROMPT = """
QUESTION REFINEMENT MODE:
1) Restate the problem in 1-2 lines.
2) Extract requirements (functional + non-functional).
3) Identify missing critical info (max 2 questions).
4) If user doesn't answer, proceed with explicit assumptions.

Return output in this exact format:
Restated problem:
Requirements:
Missing info (max 2 questions):
Assumptions (if needed):
RGC Prompt (final):
""".strip()

TEACH_AND_ASK_PROMPT = """
TEACH-AND-ASK MODE:
1) Provide 2-3 key options/facts that affect the design.
2) Ask 1-2 targeted questions that decide the correct choice.
3) Give a default recommendation if unanswered, with assumptions.

Output:
Key options:
Default recommendation:
Questions (max 2):
""".strip()

VERIFIER_PROMPT = """
COGNITIVE VERIFIER MODE:
Check the answer for:
1) Requirement coverage
2) Incorrect claims / missing assumptions
3) Security risks
4) Edge cases (at least 3)
5) Scalability/performance risks
6) Test plan adequacy

Output ONLY:
Verifier Notes:
- ...
Fix Suggestions:
- ...
""".strip()

REASONING_VISIBILITY_PROMPT = """
REASONING VISIBILITY MODE (no chain-of-thought):
Do NOT show hidden step-by-step reasoning.

Instead output:
- Key reasoning summary (3-6 bullets)
- Decision checklist
- Verification plan

Then provide final answer in this format:
Answer:
Reasoning Summary:
Trade-offs:
Failure Modes:
Production Checklist:
""".strip()

TABULAR_INSTRUCTION = """
Also include a comparison table with columns:
Option | Complexity | Latency | Cost | Dev Time | Failure Modes | Best When
Then recommend one option.
""".strip()

# ----------------------------
# Groq call helper
# ----------------------------
def ask_groq(prompt: str, temperature: float = 0.2) -> str:
    resp = client.chat.completions.create(
        model=DEFAULT_MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message.content

# ----------------------------
# Prompting modes
# ----------------------------
def few_shot(task: str) -> str:
    return f"""
Follow the format and rigor of these examples.

EXAMPLE INPUT:
Design an auth system for a SaaS app.

EXAMPLE OUTPUT:
Answer:
Use session cookies with rotating refresh tokens.
Reasoning Summary:
Improves security and revocation.
Trade-offs:
More infra.
Failure Modes:
CSRF.
Production Checklist:
Cookies, CSRF tokens.

NOW DO THIS TASK:
{task}
""".strip()

# ----------------------------
# Run pipeline (Colab-friendly)
# ----------------------------
raw_request = "Build a RAG chatbot for my college ERP. It should answer student questions and cite sources."

refined = ask_groq(f"{REFINER_PROMPT}\n\nUser request:\n{raw_request}", temperature=0.1)
teach = ask_groq(f"{TEACH_AND_ASK_PROMPT}\n\nContext:\n{refined}", temperature=0.2)
answer = ask_groq(few_shot(refined + "\n\n" + TABULAR_INSTRUCTION), temperature=0.2)
verifier = ask_groq(f"{VERIFIER_PROMPT}\n\nAnswer to verify:\n{answer}", temperature=0.1)

print("\n=== REFINED ===\n", refined)
print("\n=== TEACH & ASK ===\n", teach)
print("\n=== ANSWER ===\n", answer)
print("\n=== VERIFIER ===\n", verifier)



=== REFINED ===
 Restated problem:
Build a Retrieval-Augmented Generation (RAG) chatbot for a college Enterprise Resource Planning (ERP) system to answer student queries and provide citations. The chatbot should integrate with existing college data and knowledge sources.

Requirements:
- Functional: answer student questions, provide citations, integrate with college data and knowledge sources
- Non-functional: accuracy, reliability, scalability, maintainability, usability

Missing info (max 2 questions):
1. What is the scope of student questions the chatbot should be able to answer (e.g., academic policies, course information, student life)?
2. Are there any existing knowledge bases, databases, or APIs that the chatbot can leverage for information retrieval and citation?

Assumptions (if needed):
None at this time, awaiting user response for missing information.

RGC Prompt (final):
None at this time, awaiting user response to refine the prompt.

=== TEACH & ASK ===
 Key options:
1. *